In [23]:
import pandas as pd
from sklearn.cluster import KMeans
import json

In [24]:
property ="dielectric"
# property ="elasticity"
# property ="perovskites"
file_path = f'OFM_{property}_SparseYcluster_target_clusters50.json'
n_clusters=50
df = pd.read_csv(f"OFM_{property}_property.csv")



y = df['property'].values
excluded = ["0","1", "property"]
X = df.drop(excluded, axis=1)

In [25]:
y.shape

(4764,)

In [26]:
import numpy as np
from scipy.stats import gaussian_kde

# Generate a sample one-dimensional vector
#np.random.seed(42)
#vector = np.random.randn(100)

# Estimate the density using kernel density estimation
kde = gaussian_kde(y)

# Evaluate the estimated density at sample points
sample_points = np.linspace(y.min(), y.max(), 100)
densities = kde(sample_points)

# Print the estimated density values
# for i in range(len(sample_points)):
#     print(f"Density at sample point {sample_points[i]}: {density[i]}")

In [27]:

density_dict = {}

# Print the density values for each sample position
for i in range(len(densities)):
    density = densities[i]
    #print(f"Density at sample position {i+1}: {density}")
    
    density_dict[i] = density

#print(density_dict)

sorted_density = sorted(density_dict.items(), key=lambda x: x[1])
#print(sorted_density)
top500 = [x[0] for x in sorted_density[:500]]
X_sparse = X.iloc[top500]
print(X_sparse.shape)

(100, 1024)


In [28]:
id_map = {}

for i, index in enumerate(top500):
    id_map[i] = top500[i]
    

    

In [29]:
# Perform KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, max_iter=500, random_state=222)
kmeans.fit(X_sparse)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# Print cluster labels and centers
#print("Cluster Labels:", labels)
#print("Cluster Centers:", centers)

# Create a dictionary to store the cluster assignments
cluster = {}
#print(labels)
for i, cluster_id in enumerate(labels):
    if cluster_id in cluster:
        #cluster[int(cluster_id)].append(i)
        continue
    else:
        cluster[int(cluster_id)] = [id_map[i]]







/Users/jason/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [30]:
print(cluster)

{14: [69], 47: [70], 5: [71], 10: [67], 11: [72], 31: [66], 32: [73], 1: [65], 2: [64], 20: [75], 28: [63], 8: [76], 30: [62], 18: [60], 33: [79], 24: [59], 7: [80], 4: [58], 45: [94], 12: [57], 44: [93], 0: [30], 26: [92], 13: [96], 43: [31], 3: [46], 37: [83], 42: [29], 27: [55], 49: [32], 40: [45], 34: [28], 38: [87], 22: [50], 29: [90], 25: [98], 48: [33], 19: [44], 41: [49], 17: [99], 16: [25], 35: [34], 6: [39], 15: [52], 9: [16], 46: [15], 23: [8], 21: [6], 39: [5], 36: [3]}


In [31]:
selected_samples = []

# Print the size of each cluster
for key in sorted(cluster):
    #print(key, (cluster[key]))
    anchor_index = cluster[key][0]

    anchor_feature = X.iloc[anchor_index]
    #print(anchor_feature)
    distance_dict = {}
    for row_index in range(X.shape[0]):
        f = X.iloc[row_index]
        euclidean_distance = np.linalg.norm(anchor_feature - f)
        #print(euclidean_distance)
        
        if row_index == anchor_index:
            continue
        else:
            distance_dict[row_index] = euclidean_distance
    distance_dict = sorted(distance_dict.items(), key=lambda x: x[1])
    #print(distance_dict)
    n_added = 0
    for ck in distance_dict:
        if ck[0] not in selected_samples and n_added < 10:
            cluster[key].append(ck[0])
            selected_samples.append(ck[0])
            n_added += 1
    print(cluster[key])
    #break
for key in cluster:
    print(key, cluster[key])

[30, 626, 2446, 254, 2722, 157, 1130, 239, 2070, 2363, 1700]
[65, 857, 1168, 566, 2881, 2941, 4613, 3582, 316, 4151, 3195]
[64, 27, 77, 4624, 78, 1628, 572, 175, 1103, 724, 1507]
[46, 2614, 1478, 45, 2735, 555, 761, 2738, 246, 851, 584]
[58, 4393, 1962, 4104, 2268, 3341, 3489, 1967, 3529, 3265, 3736]
[71, 240, 524, 3682, 3632, 3698, 3997, 3506, 3070, 3848, 1705]
[39, 1319, 11, 1775, 2000, 2050, 510, 1602, 844, 1171, 2021]
[80, 2494, 2610, 2906, 1884, 1767, 1224, 976, 2313, 841, 861]
[76, 3704, 331, 4522, 3342, 85, 3357, 2939, 81, 154, 3340]
[16, 561, 1660, 2981, 2238, 1535, 896, 4592, 2806, 1441, 2937]
[67, 1877, 888, 2471, 3509, 3237, 2969, 3184, 2186, 1413, 4490]
[72, 4050, 3055, 1270, 2192, 893, 3398, 4533, 2372, 308, 2469]
[57, 4331, 924, 1977, 3473, 2595, 4685, 1667, 3296, 372, 1344]
[96, 2404, 271, 1207, 123, 1879, 3903, 2547, 4266, 4165, 41]
[69, 91, 3709, 3570, 68, 3708, 3777, 3581, 3580, 32, 4340]
[52, 2083, 4538, 351, 1750, 1783, 1403, 2197, 2280, 3973, 1334]
[25, 2147, 2276,

In [32]:
# Example: Print formulas in cluster 0
# for index in cluster[0]:
#     print(df_data.iloc[index]["formula"])
    
# Save the dictionary to a file in JSON format
with open(file_path, 'w') as file:
    json.dump(cluster, file)

In [33]:
import os

def play_beep():
    os.system("osascript -e 'beep'")

# Call the function to play the beep sound
play_beep()